In [7]:
import requests
from bs4 import BeautifulSoup

In [2]:
# 게시판 url 만들기

def get_board_url(page):
    # 페이지 번호를 입력하면 게시판 보드를 가져옴
    
    url_base = 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&page='
    url = url_base+str(page)
    
    return url
    

In [3]:
board_url = get_board_url(2)

In [5]:
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')

In [8]:
# 게시판에서 영화 게시글 link 를 가져옴   

def get_movie_links(board_url):
    # 게시판 주소 입력 -->   [ (게시글1제목, 게시글1링크), (게시글2제목, 게시글2링크)......]
    
    html = requests.get(board_url).text
    soup = BeautifulSoup(html, 'lxml')
    movies = soup.select('td.list-subject')
    
    movie_links=[]
    
    for movie in movies:
        title = movie.select('a')[0].text.strip().replace('\t','').split('댓글')[0]
        link = movie.select('a')[0]['href']
        movie_links.append((title,link))
    
    return movie_links

In [9]:
movie_links = get_movie_links(board_url)

In [10]:
print(movie_links)

[('스로어웨이즈 (The Throwaways, 2015)', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055559&page=2'), ('조지 왕의 광기 (The Madness Of King George, 1994)', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055476&page=2'), ('해난 1890 海難1890, 125 Years Memory, 2015', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055443&page=2'), ('마인드게이머스 (MindGamers, 2015)', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055427&page=2'), ('흑기사 중세로 가다 (Black Knight, 2001)', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055420&page=2'), ('고스트 앤 다크니스 (The Ghost And The Darkness, 1996)', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055415&page=2'), ('수어사이드 쇼 (This Is Your Death, 2017) The show', 'http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055319&page=2'), ('오리엔트 특급 살인사건 (Murder on the Orient Express, 1974) RESTORED.1080p.BluRay.X264-AMIABLE', 'http://cineaste.co.kr/bbs/board.php?bo_table=

In [22]:
# 게시글에 접속하여 자막 링크 주소를 가져온다

def get_sub_link(movie_link):
    # 게시글 링크 --> 자막보기 링크
    html = requests.get(movie_link).text
    soup = BeautifulSoup(html, 'lxml')
#     sub_file = soup.select('div.list-group.font-12 > a')[2]['onclick'].split("'")[1]   # 자막 주소명만 가져온다
    sub_file_candidates = soup.select('div.list-group.font-12 > a')
    for candidate in sub_file_candidates:        #자막보기 위치가 게시글마다 상이하여, 위치를 찾아준다
        if candidate.text ==' 자막보기':
            sub_file = candidate['onclick'].split("'")[1]
            break
    #thema_wrapper > div > div.container > div > div > div.panel.panel-default.view-head > div.list-group.font-12 > a:nth-child(4)
    sub_link_base = 'http://cineaste.co.kr/skin/board/apms-caption/view_caption.php?bo_table=psd_caption&fname='
    sub_link = sub_link_base+sub_file
    
    return(sub_link)


In [29]:
# 자막 링크 주소에서 자막 글을 가져온다

def get_sub_text(sub_link):
    html = requests.get(sub_link).text
    sub_text = BeautifulSoup(html, 'lxml').text.strip()
    return sub_text

In [20]:
print(movie_links[0][1])

http://cineaste.co.kr/bbs/board.php?bo_table=psd_caption&wr_id=1055559&page=2


In [23]:
sub_link = get_sub_link(movie_links[0][1])

In [24]:
sub_text = get_sub_text(sub_link)[:500]


The.Throwaways.2015.DVDRip.XviD-EVO-[Original]

<!--
P { margin-left:0pt; margin-right:0pt; margin-bottom:10pt;
    margin-top:10pt; font-size:18pt; text-align:center;
    font-family:arial; font-weight:bold; color:#f0f0f0;
    }
.KOCC { Name:Korean; lang:ko-KR; SAMIType:CC; }
-->



sub2smi by  Michael Archangel
 

“카자흐스탄”　　　　　　　　　　　
 

전 여러분이 좋아하는선량한 해커 블랙잭이에요　　　　　　　　　　
 

아침에 몹시 바빴어요
 

지하드, 악당 해커독재정권과의 1인 전쟁 때문에
 

오늘 행운의 당첨자는알 샤무크입니다
 

합법적인 뉴스 기관으로 가장해
 

지하드 테러


In [34]:
import requests
from bs4 import BeautifulSoup

BOARD_START = 1    # 씨네시티 게시판 시작 페이지
BOARD_END = 2       #  게시판 마지막 페이지

if __name__ == "__main__":
    
    movie_subs = []
    
    # 게시판 페이지 별로
    for i in range(BOARD_START, BOARD_END+1):
        borad_url = get_board_url(i)
        move_links = get_movie_links(board_url)
        
        # 게시판 글 별로
        for movie in move_links:
            title = movie[0]     #게시글 제목
            movie_link = movie[1]   # 게시글 링크 확인
            
            try:
                sub_link = get_sub_link(movie_link)   # 게시글 링크에서 자막 주소 확인,   자막보기가 없는 게시글은 pass 한다
            except:
                continue
                
            contents = get_sub_text(sub_link)     # 자막 내 정보 가져오기   (필요에 맞게 추가 전처리 필요함)
            
            movie_subs.append((title, contents[:500]))    # 테스트를 위해서 자막 글 중 500자만 가져옴. 실제할때는 없애줌. 
                                                          # 사용 목적에 따라 전처리 추가하여 가져오는것이 좋을 듯.
            print('{}의 자막을 가져왔습니다'.format(title))
    print(movie_subs)


스로어웨이즈 (The Throwaways, 2015)의 자막을 가져왔습니다
조지 왕의 광기 (The Madness Of King George, 1994)의 자막을 가져왔습니다
해난 1890 海難1890, 125 Years Memory, 2015의 자막을 가져왔습니다
마인드게이머스 (MindGamers, 2015)의 자막을 가져왔습니다
흑기사 중세로 가다 (Black Knight, 2001)의 자막을 가져왔습니다
고스트 앤 다크니스 (The Ghost And The Darkness, 1996)의 자막을 가져왔습니다
수어사이드 쇼 (This Is Your Death, 2017) The show의 자막을 가져왔습니다
오리엔트 특급 살인사건 (Murder on the Orient Express, 1974) RESTORED.1080p.BluRay.X264-AMIABLE의 자막을 가져왔습니다
끝없는 사랑.Endless.Love.1981.WEBRip.x264-RARBG의 자막을 가져왔습니다
퍼니 맨 (Funny Man 1994) Funny.Man.1994.720p.BluRay.x264-x0r의 자막을 가져왔습니다
라자루스  (The Lazarus Effect, 2015)의 자막을 가져왔습니다
위트 (Wit, 2001)의 자막을 가져왔습니다
안전은 보장할 수 없음 (Safety Not Guaranteed, 2012) LIMITED.1080p.BluRay.x264-SPARKS의 자막을 가져왔습니다
리오에서 온 사나이 (L Homme De Rio / That Man From Rio, 1964) 1080p.BluRay.x264-SADPANDA의 자막을 가져왔습니다
비스마르크호를 격침하라 (Sink The Bismarck!, 1960)의 자막을 가져왔습니다
오리엔트 특급 살인 (Murder on the Orient Express, 1974) 영문의 자막을 가져왔습니다
하얀 궁전 (White Palace. 1990) - 루이스 만도키 감독의 자막을 가져왔습니다
홍금보의 벽력대나팔 1